<a href="https://colab.research.google.com/github/lkhok22/ML-FinalProject-Walmart-Recruiting---Store-Sales-Forecasting/blob/main/model_experiment_Ensemble.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
# Install required libraries
!pip install pandas numpy matplotlib seaborn scikit-learn torch torchvision wandb pyyaml darts --quiet
import wandb
wandb.login(key="eccf2c915699fc032ad678daf0fd4b5ac60bf87c")

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [7]:
# Mount Google Drive and extract data
from google.colab import drive
import zipfile
import os
drive.mount('/content/drive')
zip_path = '/content/drive/MyDrive/ML-FinalProject/data.zip'
extract_to = '/content/walmart_data/'
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to)
for file_name in os.listdir(extract_to):
    if file_name.endswith('.zip'):
        with zipfile.ZipFile(os.path.join(extract_to, file_name), 'r') as zip_ref:
            zip_ref.extractall(extract_to)
print("✅ Extracted files:", os.listdir(extract_to))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Extracted files: ['test.csv.zip', 'stores.csv', 'sampleSubmission.csv', 'features.csv', 'test.csv', 'train.csv.zip', 'train.csv', 'sampleSubmission.csv.zip', 'features.csv.zip']


In [37]:
# Install required libraries
!pip install pandas numpy matplotlib seaborn scikit-learn torch neuralforecast wandb pyyaml --quiet

# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import torch
from neuralforecast import NeuralForecast
from neuralforecast.models import NBEATS, TFT, PatchTST, DLinear
from neuralforecast.losses.pytorch import MAE
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error
import wandb
import os
from pathlib import Path

# Set random seed for reproducibility
torch.manual_seed(42)
np.random.seed(42)

# Initialize wandb
wandb.init(project="walmart-sales-forecasting", config={
    "models": ["N-BEATS", "TFT", "PatchTST", "DLinear"],
    "horizon": 4,
    "input_size": 24,
    "max_steps": 100,  # Increased for better training
    "learning_rate": 0.001,
    "batch_size": 32
})

In [41]:
# Load data
data_path = '/content/walmart_data/'
train_df = pd.read_csv(data_path + 'train.csv')
test_df = pd.read_csv(data_path + 'test.csv')
features_df = pd.read_csv(data_path + 'features.csv')
stores_df = pd.read_csv(data_path + 'stores.csv')

# Merge datasets, keeping IsHoliday from features.csv
train_df = train_df.merge(stores_df, on='Store').merge(
    features_df, on=['Store', 'Date'], suffixes=('', '_features'), how='left'
)
test_df = test_df.merge(stores_df, on='Store').merge(
    features_df, on=['Store', 'Date'], suffixes=('', '_features'), how='left'
)

# Drop redundant IsHoliday and rename
train_df = train_df.drop(columns=['IsHoliday'], errors='ignore').rename(
    columns={'IsHoliday_features': 'IsHoliday'}
)
test_df = test_df.drop(columns=['IsHoliday'], errors='ignore').rename(
    columns={'IsHoliday_features': 'IsHoliday'}
)

# Convert Date to datetime and rename to ds
train_df['Date'] = pd.to_datetime(train_df['Date'])
test_df['Date'] = pd.to_datetime(test_df['Date'])
train_df = train_df.rename(columns={'Date': 'ds'})
test_df = test_df.rename(columns={'Date': 'ds'})

# Handle missing values in MarkDown columns
markdown_cols = ['MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5']
train_df[markdown_cols] = train_df[markdown_cols].fillna(0)
test_df[markdown_cols] = test_df[markdown_cols].fillna(0)

# Create unique_id for store-department combinations
train_df['unique_id'] = train_df['Store'].astype(str) + '_' + train_df['Dept'].astype(str)
test_df['unique_id'] = test_df['Store'].astype(str) + '_' + test_df['Dept'].astype(str)

# Prepare DataFrame for neuralforecast (requires columns: unique_id, ds, y)
train_nf = train_df[['unique_id', 'ds', 'Weekly_Sales']].rename(
    columns={'Weekly_Sales': 'y'}
)

# Add covariates
covariates_cols = ['Temperature', 'Fuel_Price', 'CPI', 'Unemployment',
                   'MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5',
                   'IsHoliday', 'Size']

# Convert IsHoliday to numeric
train_df['IsHoliday'] = train_df['IsHoliday'].astype(int)
test_df['IsHoliday'] = test_df['IsHoliday'].astype(int)

train_nf = train_nf.merge(
    train_df[['unique_id', 'ds'] + covariates_cols],
    on=['unique_id', 'ds'],
    how='left'
)

# Fill missing values
for col in covariates_cols:
    train_nf[col] = train_nf[col].fillna(train_nf[col].mean())

# Scale covariates
scaler = StandardScaler()
train_nf[covariates_cols] = scaler.fit_transform(train_nf[covariates_cols])

# Prepare test DataFrame
test_nf = test_df[['unique_id', 'ds']]
test_nf = test_nf.merge(
    test_df[['unique_id', 'ds'] + covariates_cols],
    on=['unique_id', 'ds'],
    how='left'
)

# Fill missing values in test set
for col in covariates_cols:
    test_nf[col] = test_nf[col].fillna(0)  # Fill with 0 or use mean from training

test_nf[covariates_cols] = scaler.transform(test_nf[covariates_cols])

# Filter time series with sufficient length
input_size = 24
horizon = 4
min_length = input_size + horizon
series_lengths = train_nf.groupby('unique_id').size()
valid_ids = series_lengths[series_lengths >= min_length].index
train_nf = train_nf[train_nf['unique_id'].isin(valid_ids)]
test_nf = test_nf[test_nf['unique_id'].isin(valid_ids)]

# Sort by unique_id and ds
train_nf = train_nf.sort_values(['unique_id', 'ds']).reset_index(drop=True)
test_nf = test_nf.sort_values(['unique_id', 'ds']).reset_index(drop=True)

# Calculate validation size
val_size = min(4, min(series_lengths[series_lengths >= min_length]) - input_size)

# Split data for validation
train_data = []
val_data = []

for uid in valid_ids:
    uid_data = train_nf[train_nf['unique_id'] == uid].copy()
    train_split = uid_data.iloc[:-val_size]
    val_split = uid_data.iloc[-val_size:]
    train_data.append(train_split)
    val_data.append(val_split)

train_split_df = pd.concat(train_data, ignore_index=True)
val_split_df = pd.concat(val_data, ignore_index=True)

In [42]:
# Model configurations with improved parameters
models = [
    NBEATS(
        input_size=input_size,
        h=horizon,
        max_steps=100,
        learning_rate=0.001,
        stack_types=['identity', 'trend', 'seasonality'],
        n_blocks=[2, 2, 2],  # Reduced for faster training
        mlp_units=[[256, 256], [256, 256], [256, 256]],  # Reduced size
        accelerator='cpu',
        early_stop_patience_steps=10
    ),
    TFT(
        input_size=input_size,
        h=horizon,
        max_steps=100,
        learning_rate=0.001,
        hidden_size=32,  # Reduced for faster training
        n_head=4,
        accelerator='cpu',
        early_stop_patience_steps=10
    ),
    PatchTST(
        input_size=input_size,
        h=horizon,
        max_steps=100,
        learning_rate=0.001,
        patch_len=8,  # Reduced patch length
        accelerator='cpu',
        early_stop_patience_steps=10
    ),
    DLinear(
        input_size=input_size,
        h=horizon,
        max_steps=100,
        learning_rate=0.001,
        accelerator='cpu',
        early_stop_patience_steps=10
    )
]

INFO:lightning_fabric.utilities.seed:Seed set to 1
INFO:lightning_fabric.utilities.seed:Seed set to 1
INFO:lightning_fabric.utilities.seed:Seed set to 1
INFO:lightning_fabric.utilities.seed:Seed set to 1


In [43]:
# Initialize NeuralForecast
nf = NeuralForecast(models=models, freq='W-FRI')

print("🚀 Starting model training...")

# Train models
nf.fit(
    df=train_split_df,
    static_df=None,
    val_size=val_size,
    verbose=True
)

print("✅ Model training completed!")

# Generate future DataFrame for validation predictions
val_futr_df = nf.make_future_dataframe(df=train_split_df)

# Add covariates to validation future dataframe
val_futr_df = val_futr_df.merge(
    val_split_df[['unique_id', 'ds'] + covariates_cols],
    on=['unique_id', 'ds'],
    how='left'
)

# Fill any missing covariates
val_futr_df[covariates_cols] = val_futr_df[covariates_cols].fillna(0)

# Generate validation predictions
val_predictions = nf.predict(futr_df=val_futr_df)

# Calculate validation MAE for each model
model_names = ['NBEATS', 'TFT', 'PatchTST', 'DLinear']
val_maes = {}

for model_name in model_names:
    # Get actual and predicted values
    actual = val_split_df['y'].values
    predicted = val_predictions[model_name].values

    # Calculate MAE using sklearn
    mae_score = mean_absolute_error(actual, predicted)
    val_maes[model_name] = mae_score

    # Log to wandb
    wandb.log({f"val_{model_name}_MAE": mae_score})
    print(f"Validation MAE for {model_name}: {mae_score:.4f}")

# Create ensemble prediction
ensemble_pred = val_predictions[model_names].mean(axis=1)
val_predictions['ensemble'] = ensemble_pred

# Calculate ensemble MAE
ensemble_mae = mean_absolute_error(val_split_df['y'].values, ensemble_pred.values)
val_maes['ensemble'] = ensemble_mae
wandb.log({"val_ensemble_MAE": ensemble_mae})
print(f"Validation MAE for ensemble: {ensemble_mae:.4f}")

# Now train on full dataset for final predictions
print("🚀 Training on full dataset for final predictions...")
nf_final = NeuralForecast(models=models, freq='W-FRI')
nf_final.fit(df=train_nf, static_df=None, val_size=val_size, verbose=True)

# Generate future DataFrame for test predictions
test_futr_df = nf_final.make_future_dataframe(df=train_nf)

# Add covariates to test future dataframe
test_futr_df = test_futr_df.merge(
    test_nf[['unique_id', 'ds'] + covariates_cols],
    on=['unique_id', 'ds'],
    how='left'
)

# Fill any missing covariates
test_futr_df[covariates_cols] = test_futr_df[covariates_cols].fillna(0)

# Generate final predictions
final_predictions = nf_final.predict(futr_df=test_futr_df)

# Create final ensemble
final_ensemble = final_predictions[model_names].mean(axis=1)
final_predictions['ensemble'] = final_ensemble

# Prepare submission
submission_df = test_df[['Store', 'Dept', 'ds']].copy()  # Use 'ds' instead of 'Date'
submission_df['unique_id'] = submission_df['Store'].astype(str) + '_' + submission_df['Dept'].astype(str)

# Merge predictions
submission_df = submission_df.merge(
    final_predictions[['unique_id', 'ds', 'ensemble']].rename(columns={'ensemble': 'Weekly_Sales'}),
    on=['unique_id', 'ds'],
    how='left'
)

# Create submission ID
submission_df['Id'] = (submission_df['Store'].astype(str) + '_' +
                      submission_df['Dept'].astype(str) + '_' +
                      submission_df['ds'].dt.strftime('%Y-%m-%d'))

# Final submission format
final_submission = submission_df[['Id', 'Weekly_Sales']].copy()

# Handle any missing predictions
final_submission['Weekly_Sales'] = final_submission['Weekly_Sales'].fillna(0)

# Ensure non-negative predictions
final_submission['Weekly_Sales'] = np.maximum(final_submission['Weekly_Sales'], 0)

print("📊 Submission Summary:")
print(f"Total predictions: {len(final_submission)}")
print(f"Missing predictions filled: {final_submission['Weekly_Sales'].isna().sum()}")
print(f"Mean prediction: {final_submission['Weekly_Sales'].mean():.2f}")
print(f"Std prediction: {final_submission['Weekly_Sales'].std():.2f}")

# Save submission
final_submission.to_csv('/content/walmart_submission.csv', index=False)
print("✅ Submission saved to /content/walmart_submission.csv")

# Log final metrics
wandb.log({
    "total_predictions": len(final_submission),
    "mean_prediction": final_submission['Weekly_Sales'].mean(),
    "std_prediction": final_submission['Weekly_Sales'].std()
})

# Display first few predictions
print("\n📋 First 10 predictions:")
print(final_submission.head(10))

wandb.finish()

INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.callbacks.model_summary:
  | Name         | Type          | Params | Mode 
-------------------------------------------------------
0 | loss         | MAE           | 0      | train
1 | padder_train | ConstantPad1d | 0      | train
2 | scaler       | TemporalNorm  | 0      | train
3 | blocks       | ModuleList    | 1.2 M  | train
-------------------------------------------------------
1.2 M     Trainable params
504       Non-trainable params
1.2 M     Total params
4.987     Total estimated model params size (MB)
58        Modules in train mode
0         Modules in eval mode


🚀 Starting model training...


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Predicting: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:Trainer already configured with model summary callbacks: [<class 'pytorch_lightning.callbacks.model_summary.ModelSummary'>]. Skipping setting a default `ModelSummary` callback.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


Predicting: |          | 0/? [00:00<?, ?it/s]

📊 Submission Summary:
Total predictions: 115064
Missing predictions filled: 0
Mean prediction: 1567.83
Std prediction: 8237.43
✅ Submission saved to /content/walmart_submission.csv

📋 First 10 predictions:
               Id  Weekly_Sales
0  1_1_2012-11-02  22454.871094
1  1_1_2012-11-09  21846.185547
2  1_1_2012-11-16  20994.714844
3  1_1_2012-11-23  20165.394531
4  1_1_2012-11-30      0.000000
5  1_1_2012-12-07      0.000000
6  1_1_2012-12-14      0.000000
7  1_1_2012-12-21      0.000000
8  1_1_2012-12-28      0.000000
9  1_1_2013-01-04      0.000000


mean_prediction,▁
std_prediction,▁
total_predictions,▁
val_DLinear_MAE,▁▁
val_NBEATS_MAE,▁▁
val_PatchTST_MAE,▁▁
val_TFT_MAE,▁▁
val_ensemble_MAE,▁▁
mean_prediction,1567.8335
std_prediction,8237.42871
total_predictions,115064
